In [104]:
import pandas as pd
import numpy as np


In [105]:
df = pd.read_csv('Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1 (1).csv')

In [106]:
df

,Timestamp,How old are you?,Industry,Job title,Additional context on job title,Annual salary,Other monetary comp,Currency,Currency - other,Additional context on income,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27676,2/19/2022 16:07:31,45-54,Property or Construction,Senior Project Manager,Owner rep consultant,145000,14500.0,USD,NaN,Guaranteed 10% bonus if our department meets t...,USA,California,San Francisco,11 - 20 years,11 - 20 years,College degree,Woman,Asian or Asian American
27677,2/19/2022 18:17:08,25-34,Insurance,Underwriter,NaN,67000,6700.0,USD,NaN,NaN,USA,Florida,Tampa,8 - 10 years,11 - 20 years,Master's degree,Woman,White
27678,2/19/2022 19:47:05,25-34,"Accounting, Banking & Finance",Vice Presidenti and Corporate Counsel,(in-house attorney),252000,80000.0,USD,NaN,I also receive an equity grant,US,Pennsylvania,Philadelphia,11 - 20 years,11 - 20 years,"Professional degree (MD, JD, etc.)",Woman,White
27679,2/21/2022 19:43:29,25-34,Education (Early Childhood Education),Quality Rating Specialist,Program assessor to conduct quality assessment...,62400,NaN,USD,NaN,NaN,United States,California,San Diego,11 - 20 years,8 - 10 years,College degree,Woman,Asian or Asian American


In [107]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [108]:
df.dtypes


Timestamp                                    object
How old are you?                             object
Industry                                     object
Job title                                    object
Additional context on job title              object
Annual salary                                object
Other monetary comp                         float64
Currency                                     object
Currency - other                             object
Additional context on income                 object
Country                                      object
State                                        object
City                                         object
Overall years of professional experience     object
Years of experience in field                 object
Highest level of education completed         object
Gender                                       object
Race                                         object
dtype: object

In [109]:
df.nunique()

Timestamp                                   24899
How old are you?                                7
Industry                                     1199
Job title                                   14168
Additional context on job title              6909
Annual salary                                4246
Other monetary comp                           836
Currency                                       11
Currency - other                              111
Additional context on income                 2951
Country                                       365
State                                         129
City                                         4758
Overall years of professional experience        8
Years of experience in field                    8
Highest level of education completed            6
Gender                                          5
Race                                           48
dtype: int64

To do: 
* decide if we want to only look in the US

x 1) remove timestamp <br>
x 2) set How old are you? to categorical<br>
3) refine/group industry<br>
4) refine/group job title<br>
x 5) remove Additional context on job title <br>
x 6) combine Annual salary with Other monetary comp ? and convert to float<br>
7) add Currency -other information to currency and then remove and make Currency categorical (This needs major work)<br>
x 8) remove Additional context on income <br>
9) refine country and make categorical<br>
10) clean state??<br>
11) clean City?<br>
12) make Overall years of professional experience categroical<br>
13) make Years of experience in field categorical<br>
14) refine Highest level of education completed and make categorical<br>
x 15) make gender categorical<br>
16) make race categorical and reduce categories?<br>


### Remove unnecessary columns

In [110]:
df=df.drop(['Timestamp'], axis=1)
df=df.drop(['Additional context on job title'], axis=1)
df=df.drop(['Additional context on income'], axis=1)

### Combine salary and other monetary comp then remove columns

In [111]:
df['Annual salary']=df['Annual salary'].astype('str')
df['Annual salary']=df['Annual salary'].str.replace(',','')
df['Annual salary']=df['Annual salary'].astype('float64')
df['Annual salary']=df['Annual salary'].fillna(0)
#unique_salary=df['Annual salary'].unique()
#np.sort(unique_salary)
corrected=[]
for val in df['Annual salary']:
    if val<9:
        corrected.append(val)
    elif 9<val<1000:
        corrected.append(val*1000)
    else:
        corrected.append(val)
        
df['Other monetary comp']=df['Other monetary comp'].fillna(0)
df["Total Salary"] = corrected + df['Other monetary comp']
df=df.drop(['Annual salary', 'Other monetary comp'], axis=1)


### Clean currency

In [112]:
df['Currency - other'].value_counts()

USD                       11
NOK                       10
INR                        9
DKK                        8
MYR                        8
                          ..
Norwegian kroner (NOK)     1
TRY                        1
Stock                      1
czech crowns               1
5                          1
Name: Currency - other, Length: 111, dtype: int64

In [113]:
currency_unique=df['Currency'].unique()
currency_unique

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'Other', 'CHF', 'ZAR',
       'SEK', 'HKD', 'JPY'], dtype=object)

In [114]:
df['Currency'].value_counts()

USD        23077
CAD         1651
GBP         1572
EUR          626
AUD/NZD      496
Other        145
CHF           37
SEK           37
JPY           23
ZAR           13
HKD            4
Name: Currency, dtype: int64

In [115]:
df["Currency"]=df["Currency"].astype(str)
df["Currency - other"]=df["Currency - other"].astype(str)
currency_unique=df['Currency'].unique()
currency_unique

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'Other', 'CHF', 'ZAR',
       'SEK', 'HKD', 'JPY'], dtype=object)

In [116]:
pattern = re.compile(r"(?<![A-Z])[A-Z]{3}(?![A-Z])")


In [117]:
curr=[]
for i in range(len(df["Currency"])):
    val=df["Currency"].iloc[i]
    if val == 'Other':
        result = pattern.search(df['Currency - other'].iloc[i])
        try:
            curr.append(result.group())
        except AttributeError:
            curr.append(np.nan)
    else:
        curr.append(val)

In [118]:
df['Currency']=curr
df.loc[df['Currency'] == "AUD", 'Currency'] = 'AUD/NZD'
df.loc[df['Currency'] == "NZD", 'Currency'] = 'AUD/NZD'
from collections import Counter
Counter(df['Currency'])


Counter({'USD': 23085,
         'GBP': 1574,
         'CAD': 1652,
         'EUR': 627,
         'AUD/NZD': 502,
         'INR': 10,
         nan: 37,
         'CHF': 38,
         'MYR': 8,
         'ZAR': 14,
         'SEK': 38,
         'HKD': 4,
         'NOK': 9,
         'TTD': 1,
         'BRL': 7,
         'CZK': 4,
         'DKK': 8,
         'PLN': 5,
         'TRY': 1,
         'CNY': 2,
         'ILS': 4,
         'NIS': 1,
         'RMB': 2,
         'JPY': 23,
         'SGD': 7,
         'KRW': 3,
         'THB': 2,
         'IDR': 2,
         'LKR': 1,
         'PHP': 2,
         'ARS': 3,
         'MXN': 2,
         'SAR': 1,
         'NTD': 2})

In [119]:

cur=[]
for val in df["Currency"]:
    if val in currency_unique:
        cur.append(val)
    else:
        cur.append(np.nan)
df['Currency']=cur
df['Currency'].isna().sum()


124

In [120]:
df = df[df['Currency'].notna()]
df['Currency'].isna().sum()

0

In [121]:
df=df.drop(['Currency - other'], axis=1)

In [154]:
df['Currency'].unique()

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'CHF', 'ZAR', 'SEK', 'HKD',
       'JPY'], dtype=object)

### Fitting industry categories using NLP

In [123]:
Counter(df['Industry'])

Counter({'Education (Higher Education)': 2435,
         'Computing or Tech': 4584,
         'Accounting, Banking & Finance': 1774,
         'Nonprofits': 2395,
         'Publishing': 55,
         'Education (Primary/Secondary)': 824,
         'Law': 1088,
         'Health care': 1859,
         'Utilities & Telecommunications': 350,
         'Business or Consulting': 834,
         'Art & Design': 348,
         'Government and Public Administration': 1864,
         'Public Library': 35,
         'Engineering or Manufacturing': 1662,
         'Media & Digital': 761,
         'Marketing, Advertising & PR': 1102,
         'Retail': 494,
         'Property or Construction': 380,
         'Biotechnology': 16,
         'Aerospace contracting': 1,
         'Insurance': 525,
         'Sales': 282,
         'Energy': 16,
         'Environmental regulation': 1,
         'Hospitality & Events': 254,
         'Transport or Logistics': 300,
         'Medical Devices': 3,
         'Academic research (

In [124]:
len(df['Industry'].unique())

1193

In [125]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abbyward/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/abbyward/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [126]:
def lemmatize_stemming(text):
    return (WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [127]:
doc_sample = df['Industry'].iloc[0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Education', '(Higher', 'Education)']


 tokenized and lemmatized document: 
['education', 'higher', 'education']


In [128]:
df['Industry']=df['Industry'].astype(str)
processed_text = df['Industry'].map(preprocess)
processed_text[:10]

0     [education, higher, education]
1                    [compute, tech]
2           [account, bank, finance]
3                       [nonprofits]
4           [account, bank, finance]
5     [education, higher, education]
6                          [publish]
7    [education, primary, secondary]
8                    [compute, tech]
9           [account, bank, finance]
Name: Industry, dtype: object

In [129]:
dictionary = gensim.corpora.Dictionary(processed_text)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break

0 education
1 higher
2 compute
3 tech
4 account
5 bank
6 finance
7 nonprofits
8 publish
9 primary
10 secondary
11 care
12 health
13 utilities
14 business
15 consult
16 design
17 administration
18 government
19 public
20 library


In [130]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

In [131]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_text]
bow_corpus[4310]

[(25, 1), (26, 1)]

In [132]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 25 ("advertise") appears 1 time.
Word 26 ("market") appears 1 time.


In [133]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.8685848464823792), (1, 0.4955404771168363)]


In [134]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [135]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.472*"tech" + 0.471*"compute" + 0.025*"entertainment" + 0.019*"nonprofits" + 0.003*"care" + 0.003*"health" + 0.001*"sales" + 0.001*"museum" + 0.000*"public" + 0.000*"government"
Topic: 1 
Words: 0.208*"digital" + 0.207*"media" + 0.139*"insurance" + 0.076*"hospitality" + 0.076*"events" + 0.069*"social" + 0.068*"work" + 0.035*"library" + 0.020*"environmental" + 0.020*"biotech"
Topic: 2 
Words: 0.289*"retail" + 0.215*"construction" + 0.201*"property" + 0.164*"sales" + 0.023*"real" + 0.023*"estate" + 0.013*"scientific" + 0.011*"research" + 0.008*"commercial" + 0.006*"information"
Topic: 3 
Words: 0.321*"recruitment" + 0.216*"transport" + 0.216*"logistics" + 0.058*"publish" + 0.032*"food" + 0.023*"architecture" + 0.014*"tech" + 0.014*"compute" + 0.012*"contract" + 0.011*"sciences"
Topic: 4 
Words: 0.185*"finance" + 0.184*"account" + 0.184*"bank" + 0.117*"market" + 0.115*"advertise" + 0.094*"consult" + 0.090*"business" + 0.012*"health" + 0.012*"care" + 0.001*"nonprofits"
To

In [136]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.170*"hospitality" + 0.168*"events" + 0.075*"environmental" + 0.059*"agriculture" + 0.058*"forestry" + 0.057*"research" + 0.038*"compute" + 0.038*"tech" + 0.029*"architecture" + 0.028*"science"
Topic: 1 Word: 0.325*"education" + 0.183*"higher" + 0.180*"manufacture" + 0.175*"engineer" + 0.040*"transport" + 0.040*"logistics" + 0.009*"nonprofits" + 0.005*"estate" + 0.005*"real" + 0.004*"compute"
Topic: 2 Word: 0.512*"nonprofits" + 0.178*"market" + 0.176*"advertise" + 0.016*"education" + 0.011*"engineer" + 0.011*"manufacture" + 0.009*"higher" + 0.009*"government" + 0.009*"health" + 0.009*"pharmaceuticals"
Topic: 3 Word: 0.226*"care" + 0.226*"health" + 0.105*"consult" + 0.103*"business" + 0.084*"insurance" + 0.077*"primary" + 0.077*"secondary" + 0.050*"education" + 0.009*"service" + 0.005*"energy"
Topic: 4 Word: 0.277*"government" + 0.276*"public" + 0.276*"administration" + 0.018*"account" + 0.018*"finance" + 0.018*"bank" + 0.014*"education" + 0.011*"compute" + 0.011*"tech" 

In [137]:
processed_text[4310]

['engineer', 'manufacture']

In [138]:
processed_text

0                  [education, higher, education]
1                                 [compute, tech]
2                        [account, bank, finance]
3                                    [nonprofits]
4                        [account, bank, finance]
                           ...                   
27676                    [property, construction]
27677                                 [insurance]
27678                    [account, bank, finance]
27679    [education, early, childhood, education]
27680                       [political, campaign]
Name: Industry, Length: 27557, dtype: object

In [139]:
words=processed_text.apply(pd.Series).stack().reset_index(drop=True)

In [140]:
type(words)

pandas.core.series.Series

In [141]:
len(words.unique())

655

In [147]:
df['words']=processed_text

In [142]:
d = {
    'education': ['education','instructional','interpretation', 'coach',  'print','higher','primary', 'restoration','secondary', 'educational', 'early','graduate', 'childhood','college','publish', 'edtech','academic', 'research','literature', 'university','teacher','museum','study', 'learn','museums','publications','translation','instructor','arts' ,'university','student','learn','school','archive','textbook','educ','publishibg'],
    'tech':['compute', 'internet','tech','digital', 'ecommerce','aerospace', 'virtual','technology', 'software','saas','cybersecurity','technical','database','information','fintech','data','analytics','intelligence'],
    'finance':['account' ,'fund','actuarial','bank', 'invest','finance','investment','economics','wealth'],
    'nonprofit':['nonprofits','diversity', 'equity', 'inclusion', 'charity','grantwriting','americorps', 'philanthropy', 'donor', 'nonprofit','heritage','culture','foundation','cultural','preservation','fundraise' ],
    'government':[ 'government','disability','global','urban','immigration','regulatory', 'public', 'administration','library','social', 'survey','libraries','politics','relations', 'lobby','librarian','international','govt','community','policy','federal','state','political','mail','management','compliance'],
    'healthcare':['health', 'probiotics','care', 'medcomms','medical','hospital','biopharma','pharmaceuticals','medicine','pharma','pharmaceutical','healthcare','clinical','preclinical','biomedical', 'pharmacy','toxicology','veterinarian','therapy', 'psychology','veterinary','therapist','cancer','drug','psychologist','hygiene'],
    'property':['utilities','appraisal', 'electrification', 'property','concrete','compliance','construction','costruction', 'architecture','real', 'estate' ,'interior' ,'architecture','house','furniture','architect','mortgage','interior','landscape'],
    'business':['business','media','digital','market','advertise', 'design','purchase','buyer',  'survey','review','fmcg','strategy','comm','outsource','payment','center','private','think','commerce','award', 'work','procurement','staff', 'background','human','association','contract','consult', 'insurance','trade','sales','logistics', 'recruitment','service','consumer','management','distribution','workforce','market','wholesale','organizational','labor','executive', 'professional','corporate','procurement','operations','product','administrative','legal','train','union','communications','unions','office','counsel','consultant','consultants','organization', 'logistics',  'compliance',  'plan', 'assist','accessibility'],
    'manufacturing':['engineer','inspection','destructive','quality', 'manufacture', 'distribute', 'production','supply','warehouse','distribution','chemical','chemicals', 'operations','industrial','manufactures','machinery','equipment' ],
    'science':['biotechnology','bioinformatics','aerospace', 'stem', 'research','biotech','space','sciences', 'science','ecology','scientific','scientist','chemistry','biological','bioscience','bitech','laboratory','analytical','biologist','animal'],
    'environment':['energy','exploration', 'park', 'zoos', 'environmental', 'conservation','regulation','archaeology','archaeologist','resource','renewable', 'sustainability','renewables','environment','natural'],
    'agriculture':['agriculture','maritime', 'forestry','mine','cannabis','geospatial','land','geologist','outdoor','horticulture','petroleum','fisherman','oilfield'],
    'law enforcement and military':['enforcement', 'protective','security','soldier','defense','protection','forensics','military'],
    'transportation':['auto','automotive','repair','truck', 'aviation','transport','transportation','airline'],
    'entertainment':['music','perform','entertainment','animation','leisure','sport','video','game','journalism','fitness','theatre','theater','film','music','video','perform'],
    'self-employed':[ 'gamble','retire','entrepreneur','travel','self'],
    'service':['food','storage','language','clothe','facilities', 'maintenance', 'clean','foodservice','beverage','drink', 'beer','beauty','cosmetology','housekeeper','cook','landscape','funeral','retail','hospitality','tourism','restaurant','restaurants','apparel','laundry','wine','brew','waste','customer','fashion','tailor','gyms','auction','camp','pest','coffee','janitorial','grocery','groom'],
    'religion':['religion','church', 'ministry','religious','faith', 'spirituality','clergy','chaplain'],
    'family':['childcare','daycare','children','household','parent','child','caregiver','caretaker','family','sitter']
}

In [148]:
topic=[]
for industry in df['words']:
    for word in industry:
        indust=[key for key, value in d.items() if word in value]

    topic.append(indust)

In [149]:
topic

[['education'],
 ['tech'],
 ['finance'],
 ['nonprofit'],
 ['finance'],
 ['education'],
 ['education'],
 ['education'],
 ['tech'],
 ['finance'],
 ['nonprofit'],
 ['education'],
 ['finance'],
 ['finance'],
 ['healthcare'],
 ['property'],
 ['business'],
 ['business'],
 ['business'],
 ['education'],
 ['healthcare'],
 ['nonprofit'],
 ['nonprofit'],
 ['government'],
 ['government'],
 ['education'],
 ['nonprofit'],
 ['nonprofit'],
 ['government'],
 ['manufacturing'],
 ['nonprofit'],
 ['tech', 'business'],
 ['finance'],
 ['finance'],
 ['nonprofit'],
 ['education'],
 ['tech', 'business'],
 ['finance'],
 ['government'],
 ['finance'],
 ['manufacturing'],
 ['business'],
 ['government'],
 ['tech'],
 ['tech'],
 ['nonprofit'],
 ['tech'],
 ['tech'],
 ['manufacturing'],
 ['service'],
 ['education'],
 ['education'],
 ['business'],
 ['tech'],
 ['tech', 'business'],
 ['manufacturing'],
 ['government'],
 ['business'],
 ['business'],
 ['government'],
 ['education'],
 ['government'],
 ['property'],
 ['educat

In [150]:
df['topic']=topic

In [151]:
indust=[]
for line in df['topic']:
    if len(line)==0:
        indust.append(np.nan)
    else:
        try:
            indust.append(','.join(map(str, line)))
        except TypeError:
            indust.append(np.nan)
df['topic']=indust
df

,How old are you?,Industry,Job title,Currency,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race,Total Salary,words,topic
0,25-34,Education (Higher Education),Research and Instruction Librarian,USD,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White,55000.0,"[education, higher, education]",education
1,25-34,Computing or Tech,Change & Internal Communications Manager,GBP,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,58600.0,"[compute, tech]",tech
2,25-34,"Accounting, Banking & Finance",Marketing Specialist,USD,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,34000.0,"[account, bank, finance]",finance
3,25-34,Nonprofits,Program Manager,USD,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,65000.0,[nonprofits],nonprofit
4,25-34,"Accounting, Banking & Finance",Accounting Manager,USD,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,67000.0,"[account, bank, finance]",finance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27676,45-54,Property or Construction,Senior Project Manager,USD,USA,California,San Francisco,11 - 20 years,11 - 20 years,College degree,Woman,Asian or Asian American,159500.0,"[property, construction]",property
27677,25-34,Insurance,Underwriter,USD,USA,Florida,Tampa,8 - 10 years,11 - 20 years,Master's degree,Woman,White,73700.0,[insurance],business
27678,25-34,"Accounting, Banking & Finance",Vice Presidenti and Corporate Counsel,USD,US,Pennsylvania,Philadelphia,11 - 20 years,11 - 20 years,"Professional degree (MD, JD, etc.)",Woman,White,332000.0,"[account, bank, finance]",finance
27679,25-34,Education (Early Childhood Education),Quality Rating Specialist,USD,United States,California,San Diego,11 - 20 years,8 - 10 years,College degree,Woman,Asian or Asian American,62400.0,"[education, early, childhood, education]",education


In [152]:
df['topic'].value_counts()

tech                            4852
business                        4504
education                       3615
nonprofit                       2537
government                      2226
healthcare                      2147
finance                         1857
manufacturing                   1851
property                         861
tech,business                    793
service                          674
entertainment                    299
science                          167
education,science                163
agriculture                      162
law enforcement and military      69
environment                       63
transportation                    31
government,business               24
business,manufacturing            22
religion                          18
family                            11
government,property,business      10
self-employed                      5
tech,science                       4
property,service                   3
Name: topic, dtype: int64

In [153]:
df=df.drop(['Industry'], axis=1)

## Currency exchange to USD

In [156]:
df['Currency'].unique()

array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'CHF', 'ZAR', 'SEK', 'HKD',
       'JPY'], dtype=object)

In [164]:
exchange ={1:'USD',
           1.3210938:'GBP',
           0.78664093:'CAD',
           1.0873256:'EUR',
           0.74139988:'AUD/NZD',
           1.0861636:'CHF',
           0.06495109:'ZAR',
           0.10061237:'SEK',
           0.12794878:'HKD',
           0.0087003299:'JPY'
          }

In [179]:
convert=[]
for val in df['Currency']:
    convert.append([key for key, value in exchange.items() if val in value][0])

In [180]:
convert

[1,
 1.3210938,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.78664093,
 1.3210938,
 1,
 1,
 1,
 1,
 1,
 1,
 0.78664093,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.3210938,
 1,
 1,
 1,
 1,
 1,
 1,
 0.78664093,
 0.78664093,
 1,
 1,
 1,
 1,
 0.78664093,
 1,
 1,
 1,
 1,
 1.3210938,
 1,
 0.78664093,
 1,
 0.78664093,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.3210938,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.3210938,
 1,
 1,
 1,
 0.78664093,
 1,
 1,
 1,
 0.78664093,
 1,
 1,
 1,
 1.3210938,
 1,
 1,
 1,
 1,
 1,
 1,
 0.78664093,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.78664093,
 1,
 1,
 1,
 1.3210938,
 1,
 1.3210938,
 1.0873256,
 1,
 1,
 0.78664093,
 1,
 1,
 1,
 1,
 0.78664093,
 1.3210938,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.74139988,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.3210938,
 1.0873256,
 1,
 1.3210938,
 1,
 1,
 1,
 1.3210938,
 1,
 1.3210938,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.0873256,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1.3210938,
 1,
 1,
 1,
 1,
 1

In [181]:
df['convert']=convert

In [184]:
df['convert']=df['convert'].astype(float)
df["Total Salary"]=df["Total Salary"].astype(float)
df["Salary (USD)"] = df["convert"] * df["Total Salary"]

In [185]:
df

,How old are you?,Job title,Currency,Country,State,City,Overall years of professional experience,Years of experience in field,Highest level of education completed,Gender,Race,Total Salary,words,topic,convert,Salary (USD)
0,25-34,Research and Instruction Librarian,USD,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White,55000.0,"[education, higher, education]",education,1.000000,55000.00000
1,25-34,Change & Internal Communications Manager,GBP,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,58600.0,"[compute, tech]",tech,1.321094,77416.09668
2,25-34,Marketing Specialist,USD,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,34000.0,"[account, bank, finance]",finance,1.000000,34000.00000
3,25-34,Program Manager,USD,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,65000.0,[nonprofits],nonprofit,1.000000,65000.00000
4,25-34,Accounting Manager,USD,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,67000.0,"[account, bank, finance]",finance,1.000000,67000.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27676,45-54,Senior Project Manager,USD,USA,California,San Francisco,11 - 20 years,11 - 20 years,College degree,Woman,Asian or Asian American,159500.0,"[property, construction]",property,1.000000,159500.00000
27677,25-34,Underwriter,USD,USA,Florida,Tampa,8 - 10 years,11 - 20 years,Master's degree,Woman,White,73700.0,[insurance],business,1.000000,73700.00000
27678,25-34,Vice Presidenti and Corporate Counsel,USD,US,Pennsylvania,Philadelphia,11 - 20 years,11 - 20 years,"Professional degree (MD, JD, etc.)",Woman,White,332000.0,"[account, bank, finance]",finance,1.000000,332000.00000
27679,25-34,Quality Rating Specialist,USD,United States,California,San Diego,11 - 20 years,8 - 10 years,College degree,Woman,Asian or Asian American,62400.0,"[education, early, childhood, education]",education,1.000000,62400.00000


In [187]:
df=df.drop(['Total Salary','convert'], axis=1)